In [6]:
# taa_vix_filter_vectorbt_corrected_v2.py
# Replicates the "Timing Leveraged Equity Exposure in a TAA Model" overlay
# using Bloomberg data (xbbg) and vectorbt (OSS), with robust index handling.

import pandas as pd
import numpy as np
from xbbg import blp
import vectorbt as vbt

# -------------------
# Helpers
# -------------------
def extract_field(df, field='PX_LAST'):
    """Return a 2D DataFrame of the requested field with tickers as columns.
    Works whether xbbg.bdh returned single-field (flat) or multi-field (MultiIndex) columns."""
    cols = df.columns
    if isinstance(cols, pd.MultiIndex):
        for lvl in range(cols.nlevels):
            if field in cols.get_level_values(lvl):
                out = df.xs(field, axis=1, level=lvl)
                if isinstance(out, pd.Series):
                    out = out.to_frame()
                return out
        raise KeyError(f"Field {field!r} not found in MultiIndex columns: {cols.names}")
    else:
        return df

def ensure_dtindex(obj):
    """Coerce Series/DataFrame index to tz-naive DatetimeIndex, sorted, drop unparseable."""
    if isinstance(obj, (pd.Series, pd.DataFrame)):
        out = obj.copy()
        out.index = pd.to_datetime(out.index, errors='coerce')
        out = out[out.index.notna()]
        try:
            if getattr(out.index, 'tz', None) is not None:
                out.index = out.index.tz_localize(None)
        except Exception:
            pass
        return out.sort_index()
    return obj

def ann_vol(daily_ret, periods=252):
    return daily_ret.std() * np.sqrt(periods)

def vol_target_path(nav_series, target_vol=0.10, periods=252):
    """Post-hoc scale a NAV path to target annualized vol for comparability (reporting only)."""
    nav_series = nav_series.dropna()
    if len(nav_series) < 2:
        return nav_series.copy()
    ret = nav_series.pct_change().fillna(0.0)
    current_vol = ann_vol(ret, periods=periods)
    if current_vol == 0 or np.isnan(current_vol):
        return nav_series.copy()
    scale = target_vol / current_vol
    scaled_ret = ret * scale
    scaled_nav = (1 + scaled_ret).cumprod()
    scaled_nav *= nav_series.iloc[0] / scaled_nav.iloc[0]
    return scaled_nav

# -------------------
# Parameters
# -------------------
start = '2013-01-01'
end   = '2025-09-30'           # article span
fees  = 0.0005                 # 5 bps per trade
use_cash_proxy = True          # False = assume 0% cash return
cash_ticker = 'BIL US Equity'  # or 'SHV US Equity'
fallback_weight = 0.30         # weight of the fallback sleeve
robust = False                 # True -> 10/15/20-day voting logic

# -------------------
# Bloomberg tickers & data
# -------------------
px_tickers = [
    'SPY US Equity','SPXL US Equity','TLT US Equity','GLD US Equity',
    'DBC US Equity','UUP US Equity','BTAL US Equity'
]
if use_cash_proxy:
    px_tickers = px_tickers + [cash_ticker]

prices_raw = blp.bdh(tickers=px_tickers, flds='PX_LAST', start_date=start, end_date=end)
prices = extract_field(prices_raw, 'PX_LAST')

vix_raw = blp.bdh(tickers='VIX Index', flds='PX_LAST', start_date=start, end_date=end)
vix = extract_field(vix_raw, 'PX_LAST').squeeze()
vix.name = 'VIX'

# Standardize column names
rename_map = {
    'SPY US Equity':'SPY',
    'SPXL US Equity':'SPXL',
    'TLT US Equity':'TLT',
    'GLD US Equity':'GLD',
    'DBC US Equity':'DBC',
    'UUP US Equity':'UUP',
    'BTAL US Equity':'BTAL',
    'BIL US Equity':'BIL',
    'SHV US Equity':'SHV'
}
prices = prices.rename(columns={k:v for k,v in rename_map.items() if k in prices.columns})
cash_short = rename_map.get(cash_ticker, cash_ticker) if use_cash_proxy else None

# Ensure datetime indices
prices = ensure_dtindex(prices)
vix    = ensure_dtindex(vix)

# Join and clean
data = ensure_dtindex(prices.join(vix, how='inner'))
data = data.loc[~data.index.duplicated(keep='first')].dropna(how='all')

# -------------------
# Signal: RV vs VIX
# -------------------
ret_spy = data['SPY'].pct_change()

def realized_vol(series, win):
    return series.rolling(win).std() * np.sqrt(252)

if robust:
    wins = [10, 15, 20]
    rv_df = pd.concat({w: realized_vol(ret_spy, w) for w in wins}, axis=1).dropna()
    vix_aligned = data['VIX'].reindex(rv_df.index)
    rv_gt_vix = (rv_df > (vix_aligned / 100).values.reshape(-1,1)).astype(int)
    votes = rv_gt_vix.sum(axis=1)
    spxl_alloc_frac = votes.map({0: 1.0, 1: 2/3, 2: 1/3, 3: 0.0})
    signal_series = spxl_alloc_frac.astype(float)
else:
    rv20 = realized_vol(ret_spy, 20).dropna()
    vix_aligned = data['VIX'].reindex(rv20.index)
    signal_series = (rv20 < (vix_aligned / 100)).astype(float)

signal_series = ensure_dtindex(signal_series).astype(float)
signal_m = signal_series.resample('M').last()

# -------------------
# Fallback sleeve weights {SPXL, CASH}
# -------------------
weights_fallback = pd.DataFrame(index=signal_m.index, columns=['SPXL','CASH'], dtype=float)
weights_fallback['SPXL'] = signal_m.values
weights_fallback['CASH'] = 1.0 - weights_fallback['SPXL']

# -------------------
# Full portfolio weights (placeholder sleeves + fallback)
# -------------------
sleeves = ['TLT','GLD','DBC','UUP','BTAL']
n_sleeves = len(sleeves)

tradables = ['SPXL','TLT','GLD','DBC','UUP','BTAL']
if use_cash_proxy:
    tradables.append(cash_short)
tradables = [c for c in tradables if c in data.columns]

base_w = pd.DataFrame(0.0, index=weights_fallback.index, columns=tradables)

# Equal-weight the non-fallback sleeves within (1 - fallback_weight)
if n_sleeves > 0:
    for s in sleeves:
        if s in base_w.columns:
            base_w[s] = (1 - fallback_weight) / n_sleeves

# Split fallback between SPXL and Cash per the signal
if 'SPXL' in base_w.columns:
    base_w['SPXL'] = fallback_weight * weights_fallback['SPXL']
if use_cash_proxy:
    if cash_short not in base_w.columns:
        raise ValueError(f"Cash proxy '{cash_short}' missing from price matrix.")
    base_w[cash_short] = fallback_weight * weights_fallback['CASH']

# Baseline (no VIX filter): fallback fully in SPXL
base_w_nofilter = base_w.copy()
if 'SPXL' in base_w_nofilter.columns:
    base_w_nofilter['SPXL'] = fallback_weight * 1.0
if use_cash_proxy and (cash_short in base_w_nofilter.columns):
    base_w_nofilter[cash_short] = 0.0

# -------------------
# Reindex weights to daily & build price matrix
# -------------------
price_cols = [c for c in base_w.columns if c in data.columns]
price_mat = data[price_cols].copy()

w_filter_daily   = base_w.reindex(price_mat.index).ffill().fillna(0.0)
w_nofilter_daily = base_w_nofilter.reindex(price_mat.index).ffill().fillna(0.0)

# Sanity: ensure same columns & order
w_filter_daily = w_filter_daily[price_mat.columns]
w_nofilter_daily = w_nofilter_daily[price_mat.columns]

# -------------------
# VectorBT (OSS): use from_orders with targetpercent
# -------------------
pf_filter = vbt.Portfolio.from_orders(
    close=price_mat,
    size=w_filter_daily,              # target % weights
    size_type='targetpercent',
    fees=fees,
    fixed_fees=0.0,
    slippage=0.0,
    cash_sharing=True,
    init_cash=1_000_000,
    freq='D'
)

pf_nofilter = vbt.Portfolio.from_orders(
    close=price_mat,
    size=w_nofilter_daily,
    size_type='targetpercent',
    fees=fees,
    fixed_fees=0.0,
    slippage=0.0,
    cash_sharing=True,
    init_cash=1_000_000,
    freq='D'
)

# -------------------
# Stats & 10% vol-scaled NAVs
# -------------------
print("=== With VIX Filter ===")
print(pf_filter.stats())
print("\n=== Baseline (No Filter) ===")
print(pf_nofilter.stats())

nav_filter   = pf_filter.value()
nav_nofilter = pf_nofilter.value()

nav_filter_10 = vol_target_path(nav_filter, target_vol=0.10)
nav_nofilt_10 = vol_target_path(nav_nofilter, target_vol=0.10)

out = pd.DataFrame({
    'NAV_Filter'     : nav_filter,
    'NAV_NoFilter'   : nav_nofilter,
    'NAV_Filter_10%' : nav_filter_10,
    'NAV_NoFilt_10%' : nav_nofilt_10,
}).dropna()
out.to_csv('taa_vix_filter_navs.csv', index=True)

print("\nSaved NAV series to 'taa_vix_filter_navs.csv'")


C:\Users\Eddy\AppData\Local\Temp\ipykernel_32736\794715721.py:133: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  signal_m = signal_series.resample('M').last()


=== With VIX Filter ===
Start                                  2013-01-02 00:00:00
End                                    2025-09-30 00:00:00
Period                                  3206 days 00:00:00
Start Value                                      1000000.0
End Value                                   3175778.916753
Total Return [%]                                217.577892
Benchmark Return [%]                            381.054944
Max Gross Exposure [%]                               100.0
Total Fees Paid                               34751.572498
Max Drawdown [%]                                   32.8233
Max Drawdown Duration                    306 days 00:00:00
Total Trades                                          8422
Total Closed Trades                                   8416
Total Open Trades                                        6
Open Trade PnL                               207124.505705
Win Rate [%]                                       59.0423
Best Trade [%]                  

In [11]:
# exact_taa_vix_filter_xbbg.py
# Exact replication of "Timing Leveraged Equity Exposure in a TAA Model"
# Only change: data source is Bloomberg via xbbg. All warnings suppressed.

import warnings; warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from xbbg import blp
from scipy.stats import skew, kurtosis

# ========================
# Citations / Rules source
# ========================
# - Monthly rebalance; use only info up to prior trading day; 5 bps per trade;
#   fallback toggles SPXL<->Cash using RV(20d) vs VIX; other sleeves unchanged (TLT, GLD, DBC, UUP, BTAL); report at 10% vol.
#   Source: user-provided PDF summary. :contentReference[oaicite:1]{index=1}

# ========================
# Parameters
# ========================
START = '2013-01-01'
END   = '2025-09-30'
FEES  = 0.0005          # 5 bps per trade :contentReference[oaicite:2]{index=2}
ROBUST = False          # True -> 10/15/20 voting variant :contentReference[oaicite:3]{index=3}
FALLBACK_WEIGHT = 0.30  # portion of portfolio allocated to fallback sleeve (your TAA uses its own; plug exact below)
RF_ANNUAL = 0.0         # risk-free for Sharpe/Sortino reporting
INIT_NAV = 1_000_000.0
TARGET_VOL = 0.10       # for reporting-only scaling (not a trading rule) :contentReference[oaicite:4]{index=4}

# ================
# Helper functions
# ================
def extract_field(df, field='PX_LAST'):
    cols = df.columns
    if isinstance(cols, pd.MultiIndex):
        for lvl in range(cols.nlevels):
            if field in cols.get_level_values(lvl):
                out = df.xs(field, axis=1, level=lvl)
                return out if isinstance(out, pd.DataFrame) else out.to_frame()
        raise KeyError(f"{field} not found")
    return df

def ensure_dtindex(obj):
    if isinstance(obj, (pd.Series, pd.DataFrame)):
        out = obj.copy()
        out.index = pd.to_datetime(out.index, errors='coerce')
        out = out[out.index.notna()].sort_index()
        try:
            if getattr(out.index, 'tz', None) is not None:
                out.index = out.index.tz_localize(None)
        except Exception:
            pass
        return out
    return obj

def realized_vol(dret, win):
    # annualized realized volatility from daily returns
    return dret.rolling(win).std() * np.sqrt(252)

def compute_drawdowns(nav):
    peak = nav.cummax()
    dd = nav / peak - 1.0
    max_dd = float(dd.min())
    # duration
    dur = 0; max_dur = 0
    for v in dd.values:
        if v < 0: 
            dur += 1; max_dur = max(max_dur, dur)
        else:
            dur = 0
    # dates
    trough_date = dd.idxmin()
    pre_peak = nav.loc[:trough_date]
    peak_date = pre_peak.idxmax()
    recov_date = None
    post = nav.loc[trough_date:]
    if not post.empty:
        cm = post.cummax()
        x = cm[cm >= nav.loc[peak_date]]
        recov_date = x.index[0] if len(x) else None
    return max_dd, max_dur, dd, peak_date, trough_date, recov_date

def annualize_return(nav, periods_per_year=252):
    ret = nav.pct_change().dropna()
    if ret.empty: return np.nan
    total = nav.iloc[-1] / nav.iloc[0] - 1.0
    years = len(ret) / periods_per_year
    return (1 + total)**(1/years) - 1 if years > 0 else np.nan

def ann_vol(daily_ret, periods=252):
    return daily_ret.std(ddof=0) * np.sqrt(periods)

def downside_dev(daily_ret, mar=0.0, periods=252):
    dn = np.minimum(daily_ret - mar/periods, 0.0)
    return np.sqrt((dn**2).mean()) * np.sqrt(periods)

def stats_panel(nav, rf=0.0):
    nav = nav.dropna()
    ret = nav.pct_change().dropna()
    if ret.empty:
        return {}, pd.Series(dtype=float)
    cagr = annualize_return(nav)
    vol = ann_vol(ret)
    sharpe = np.nan if vol == 0 else ((ret.mean()*252 - rf)/vol)
    ddev = downside_dev(ret)
    sortino = np.nan if ddev == 0 else ((ret.mean()*252 - rf)/ddev)
    mdd, mdd_len, dd_series, dd_peak, dd_trough, dd_recov = compute_drawdowns(nav)
    calmar = np.nan if mdd == 0 else cagr/abs(mdd)
    hit_d = (ret > 0).mean()
    mret = ret.resample('ME').apply(lambda x: (1+x).prod()-1)
    yret = ret.resample('YE').apply(lambda x: (1+x).prod()-1)
    worst_m = mret.min() if not mret.empty else np.nan
    worst_y = yret.min() if not yret.empty else np.nan
    return {
        'Start'       : nav.index[0].date(),
        'End'         : nav.index[-1].date(),
        'CAGR'        : float(cagr),
        'AnnVol'      : float(vol),
        'Sharpe'      : float(sharpe),
        'Sortino'     : float(sortino),
        'MaxDD'       : float(mdd),
        'DD_Peak'     : dd_peak.date() if dd_peak is not None else None,
        'DD_Trough'   : dd_trough.date() if dd_trough is not None else None,
        'DD_Recovery' : dd_recov.date() if dd_recov is not None else None,
        'DD_DurationDays': int(mdd_len),
        'Calmar'      : float(calmar),
        'HitRate_Daily': float(hit_d),
        'Skew'        : float(skew(ret, bias=False)) if len(ret) > 2 else np.nan,
        'Kurtosis'    : float(kurtosis(ret, fisher=True, bias=False)) if len(ret) > 3 else np.nan,
        'N_Days'      : int(len(ret))
    }, dd_series

def vol_target_path(nav, target_vol=0.10):
    ret = nav.pct_change().fillna(0.0)
    v = ann_vol(ret)
    if v == 0 or np.isnan(v): return nav.copy()
    scale = target_vol / v
    scaled = (1 + ret*scale).cumprod()
    scaled *= nav.iloc[0] / scaled.iloc[0]
    return scaled

# =======================
# Load base TAA weights
# =======================
def load_base_taa_weights(me_index, sleeves):
    """
    Return a DataFrame of monthly weights for the non-fallback sleeves
    (columns = sleeves; index = true month-ends present in price data).
    Rows should sum to 1.0 - FALLBACK_WEIGHT.
    To exactly match your TAA model, replace this stub with a CSV reader that loads your historical weights.
    Example CSV format:
        date,TLT,GLD,DBC,UUP,BTAL
        2013-01-31,0.10,0.05,0.10,0.15,0.30
        ...
    """
    # --- PLACEHOLDER: equal-weight within (1 - FALLBACK_WEIGHT) ---
    w = pd.DataFrame(0.0, index=me_index, columns=sleeves)
    if len(sleeves) > 0:
        for s in sleeves:
            w[s] = (1 - FALLBACK_WEIGHT) / len(sleeves)
    return w

# ===========
# Data (BBG)
# ===========
tickers = ['SPY US Equity','SPXL US Equity','TLT US Equity','GLD US Equity','DBC US Equity','UUP US Equity','BTAL US Equity']
prices_raw = blp.bdh(tickers=tickers, flds='PX_LAST', start_date=START, end_date=END)
prices = extract_field(prices_raw, 'PX_LAST')

vix_raw = blp.bdh(tickers='VIX Index', flds='PX_LAST', start_date=START, end_date=END)
vix = extract_field(vix_raw, 'PX_LAST').squeeze(); vix.name = 'VIX'

rename = {
    'SPY US Equity':'SPY','SPXL US Equity':'SPXL','TLT US Equity':'TLT','GLD US Equity':'GLD',
    'DBC US Equity':'DBC','UUP US Equity':'UUP','BTAL US Equity':'BTAL'
}
prices = prices.rename(columns={k:v for k,v in rename.items() if k in prices.columns})

prices = ensure_dtindex(prices)
vix    = ensure_dtindex(vix)

data = ensure_dtindex(prices.join(vix, how='inner')).dropna(how='any', subset=['SPY','SPXL','VIX'])
data = data.loc[~data.index.duplicated(keep='first')]

# ==========
# Signal
# ==========
spy_ret = data['SPY'].pct_change()
if ROBUST:
    wins = [10,15,20]    # variant described in the post :contentReference[oaicite:5]{index=5}
    rv_df = pd.concat({w: realized_vol(spy_ret, w) for w in wins}, axis=1).dropna()
    vix_al = data['VIX'].reindex(rv_df.index)
    # RV in decimal; VIX in percent
    votes = (rv_df.gt((vix_al/100).values.reshape(-1,1))).sum(axis=1)  # 0..3
    # Map votes -> SPXL fraction in fallback: 0->1.0, 1->2/3, 2->1/3, 3->0.0 :contentReference[oaicite:6]{index=6}
    spxl_frac = votes.map({0:1.0, 1:2/3, 2:1/3, 3:0.0}).astype(float)
    signal_daily = spxl_frac
else:
    rv20 = realized_vol(spy_ret, 20).dropna()  # baseline uses 20d RV vs VIX :contentReference[oaicite:7]{index=7}
    vix_al = data['VIX'].reindex(rv20.index)
    signal_daily = (rv20 < (vix_al/100)).astype(float)  # 1->SPXL, 0->Cash

signal_daily = ensure_dtindex(signal_daily)
# Use only info up to T-1: shift the signal forward by 1 business day
signal_daily_t1 = signal_daily.shift(1, freq='B')
# Sample true Month-End (ME) for month-end rebalance decision
signal_me = signal_daily_t1.resample('ME').last()

# ======================================
# Build monthly target weights (0% cash)
# ======================================
sleeves = ['TLT','GLD','DBC','UUP','BTAL']
# month-ends present in price matrix
me_index = data.resample('ME').last().index

# non-fallback sleeve weights from your TAA model:
w_sleeves = load_base_taa_weights(me_index, sleeves)   # <<--- REPLACE WITH YOUR ACTUAL TAA WEIGHTS for exact replication

# assemble portfolio weights
cols = ['SPXL'] + sleeves
w_filter = pd.DataFrame(0.0, index=me_index, columns=cols)

# sleeves (sum to 1 - FALLBACK_WEIGHT)
for s in sleeves:
    if s in w_filter.columns and s in w_sleeves.columns:
        w_filter[s] = w_sleeves[s]

# fallback toggles between SPXL and 0%-cash (no cash column; residual sits uninvested)
# here we keep the full portfolio sum at 1.0 by allocating fallback entirely to SPXL or to cash (implicitly)
w_filter['SPXL'] = FALLBACK_WEIGHT * signal_me.reindex(me_index).fillna(method='ffill').fillna(0.0)

# Baseline (no filter): fallback always SPXL
w_baseline = w_filter.copy()
w_baseline['SPXL'] = FALLBACK_WEIGHT * 1.0

# ==============================
# Manual backtest (monthly ME)
# ==============================
price_mat = data[[c for c in cols if c in data.columns]].copy()
rets = price_mat.pct_change().fillna(0.0)

def run_backtest_me(pr_rets, w_me, fees=FEES, init_nav=INIT_NAV):
    # expand monthly weights to daily (piecewise-constant)
    w_daily = w_me.reindex(pr_rets.resample('ME').last().index).ffill()
    w_daily = w_daily.reindex(pr_rets.index).ffill().fillna(0.0)

    # identify rebalance days (true ME present in returns index)
    reb_days = w_daily.index.isin(w_me.index)
    prev_w = w_daily.shift(1).fillna(0.0)
    delta = (w_daily - prev_w).abs()
    turnover = delta.sum(axis=1) * reb_days.astype(int)

    nav = pd.Series(index=pr_rets.index, dtype=float)
    nav.iloc[0] = init_nav
    port_ret = pd.Series(0.0, index=pr_rets.index)

    # iterate daily; invest weights during day t that were set at end of t-1
    for i in range(1, len(pr_rets)):
        w_t = w_daily.iloc[i-1]
        gross = float(np.dot(w_t.values, pr_rets.iloc[i].values))
        fee = fees * turnover.iloc[i]  # proportional cost on rebalance days
        net = gross - fee
        port_ret.iloc[i] = net
        nav.iloc[i] = nav.iloc[i-1] * (1 + net)

    # monthly average turnover (on rebalance days)
    avg_monthly_to = turnover[reb_days].mean() if turnover[reb_days].size else 0.0
    return nav.dropna(), port_ret.dropna(), float(avg_monthly_to)

nav_f, ret_f, avg_to_f = run_backtest_me(rets, w_filter, fees=FEES, init_nav=INIT_NAV)
nav_b, ret_b, avg_to_b = run_backtest_me(rets, w_baseline, fees=FEES, init_nav=INIT_NAV)

# 10% vol-scaled paths for reporting (as in the post)
nav_f_10 = vol_target_path(nav_f, TARGET_VOL)
nav_b_10 = vol_target_path(nav_b, TARGET_VOL)

# ==================
# Stats & printout
# ==================
def pretty(d, title):
    print(f"\n=== {title} ===")
    for k, v in d.items():
        if isinstance(v, float):
            if 'DD' in k or 'Worst' in k:
                print(f"{k:22s}: {v: .2%}")
            else:
                print(f"{k:22s}: {v: .4f}")
        else:
            print(f"{k:22s}: {v}")

stats_f, _ = stats_panel(nav_f, rf=RF_ANNUAL)
stats_b, _ = stats_panel(nav_b, rf=RF_ANNUAL)
stats_f['AvgMonthlyTurnover'] = avg_to_f
stats_b['AvgMonthlyTurnover'] = avg_to_b

stats_f10, _ = stats_panel(nav_f_10, rf=RF_ANNUAL)
stats_b10, _ = stats_panel(nav_b_10, rf=RF_ANNUAL)

pretty(stats_f,  "WITH VIX FILTER (raw)")
pretty(stats_b,  "BASELINE (raw)")
pretty(stats_f10,"WITH VIX FILTER (10% vol-scaled)")
pretty(stats_b10,"BASELINE (10% vol-scaled)")

# Save artifacts if desired
out = pd.DataFrame({
    'NAV_Filter'     : nav_f,
    'NAV_NoFilter'   : nav_b,
    'NAV_Filter_10%' : nav_f_10,
    'NAV_NoFilt_10%' : nav_b_10
}).dropna(how='all')
out.to_csv('exact_taa_vix_filter_navs.csv')
print("\nSaved NAV series -> exact_taa_vix_filter_navs.csv")



=== WITH VIX FILTER (raw) ===
Start                 : 2013-01-02
End                   : 2025-09-30
CAGR                  :  0.0987
AnnVol                :  0.1331
Sharpe                :  0.7743
Sortino               :  1.0712
MaxDD                 : -31.54%
DD_Peak               : 2020-02-19
DD_Trough             : 2020-03-23
DD_Recovery           : 2021-04-13
DD_DurationDays       : 288
Calmar                :  0.3130
HitRate_Daily         :  0.5473
Skew                  : -0.8358
Kurtosis              :  29.9746
N_Days                : 3205
AvgMonthlyTurnover    :  0.0759

=== BASELINE (raw) ===
Start                 : 2013-01-02
End                   : 2025-09-30
CAGR                  :  0.1202
AnnVol                :  0.1527
Sharpe                :  0.8202
Sortino               :  1.1479
MaxDD                 : -31.54%
DD_Peak               : 2020-02-19
DD_Trough             : 2020-03-23
DD_Recovery           : 2020-07-22
DD_DurationDays       : 466
Calmar                :  0.38